In [8]:
from bs4 import BeautifulSoup
import urllib.parse as urlparse

In [33]:
url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService"
params = {'solYear' : '2019', 'solMonth' :'03'}
operation = "getRestDeInfo"
mykey = 'ac%2FSOW4KzOFKdu0z01iEVWGZd4TBl1MyiR04%2FfYmADthCjJBEyL73pewbapUk94Gm1%2FqtzMCban3C%2BpqU8c7ew%3D%3D'

In [34]:
params = urlparse.urlencode(params)
params

'solYear=2019&solMonth=03'

In [35]:
rq_query = url +'/' + operation + '?' + params + "&serviceKey=" + mykey
rq_query

'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?solYear=2019&solMonth=03&serviceKey=ac%2FSOW4KzOFKdu0z01iEVWGZd4TBl1MyiR04%2FfYmADthCjJBEyL73pewbapUk94Gm1%2FqtzMCban3C%2BpqU8c7ew%3D%3D'

In [36]:
response = requests.get(rq_query)
response

<Response [200]>

In [37]:
dom = BeautifulSoup(response.content, "html.parser")
dom

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultcode>00</resultcode><resultmsg>NORMAL SERVICE.</resultmsg></header><body><items><item><datekind>01</datekind><datename>삼일절</datename><isholiday>Y</isholiday><locdate>20190301</locdate><seq>1</seq></item></items><numofrows>10</numofrows><pageno>1</pageno><totalcount>1</totalcount></body></response>

In [40]:
datename = dom.find_all("item")
datename[0].datename.string

'삼일절'

In [41]:
date = []
datename = []
for year in range(2016, 2020):
    year = str(year)
    
    for month in range(1, 13):
        if month < 10:
            month = "0" + str(month)
        else:
            month = str(month)
            
        params = {'solYear' : year, 'solMonth' : month}
        rq_query = url +'/' + operation + '?' + urlparse.urlencode(params) + "&serviceKey=" + mykey    
        response = requests.get(rq_query) 
        dom = BeautifulSoup(response.content, "html.parser")
        
        items = dom.find_all("item")
        for item in items:
            date.append(item.locdate.string)
            datename.append(item.datename.string)
            


In [42]:
holiday_df= pd.DataFrame({"date": date, "datename": datename})
holiday_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      70 non-null     object
 1   datename  70 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [43]:
holiday_df.head()

,date,datename
0,20160101,신정
1,20160207,설날
2,20160208,설날
3,20160209,설날
4,20160210,대체공휴일


In [45]:
holiday_df.to_pickle("./holiday_df.pkl")